<a href="https://colab.research.google.com/github/marek-bardonski/airev-advanced-workshops/blob/master/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
print ("Hello world!")

Hello world!
